In [ ]:
import os
import datetime
from nbmultitask import ThreadWithLogAndControls
from IPython.display import clear_output
from time import sleep
import builtins
from time import sleep


#Functions for unit calculations (Before intersect with fishnet) => Get densities
def add_and_calculate_geodesic_unit(fc):
    #Create the temporary id for joining fields later
    arcpy.AddField_management(fc,'tempid','LONG')
    arcpy.CalculateField_management(fc,"tempid",'!OBJECTID!','PYTHON')
    #Calculate the area(land+water) by GEODESIC method => Kytt recommended
    arcpy.management.CalculateGeometryAttributes(fc, "GEODESIC_AREA AREA_GEODESIC", "", "SQUARE_METERS")
def get_and_calculate_water(fc, waterfc, state):
    #Create water mask to calculate the water area
    water_mask = arcpy.analysis.Clip(fc, waterfc, f"in_memory/{state}water_clipped")
    arcpy.management.CalculateGeometryAttributes(water_mask, "GEODESIC_WATER AREA_GEODESIC", "", "SQUARE_METERS")
    #join the GEODESIC_WATER field back to the boundary feature based on tempid
    arcpy.management.JoinField(fc, "tempid", water_mask, "tempid", ["GEODESIC_WATER"])
def calculate_land_area(fc):
    #Add field GEODESIC_LAND and calculate by AREA - WATER => LAND
    arcpy.AddField_management(fc, "GEODESIC_LAND", "FLOAT")
    with arcpy.da.UpdateCursor(fc, ["GEODESIC_AREA", "GEODESIC_WATER", "GEODESIC_LAND"]) as cursor:
            for row in cursor:
                if row[1] == None:
                    row[1] = 0 #After join there are Null values so I make it to be 0
                    row[2] = row[0] #GEODESIC_LAND is being calculated
                    cursor.updateRow(row)
                else:
                    row[2] = row[0] - row[1] 
                    cursor.updateRow(row)
def calculate_density(fc):
    #Get all the population related fields
    fields = [field.name for field in arcpy.ListFields(fc)]
    new_fields = fields[35:92] + fields[93:-6]
    #Calculate the densities for all those fields
    for variableField in new_fields:
        arcpy.AddField_management(fc,f"{variableField}_DENS",'DOUBLE')
        exp = f"get_dens(!{variableField}!, !GEODESIC_LAND!)"

        codeblock = """def get_dens(field, land):
                            if (land == 0 and field >= 0):
                                return 0
                            else:
                                return field/land

        """

        arcpy.management.CalculateField(fc,f"{variableField}_DENS",exp,"PYTHON", codeblock)
        
#Functions for PIXEL calculations (After intersect with fishnet) => Get counts
def add_and_calculate_geodesic_pixel(intersectFc):
    arcpy.management.AddField(intersectFc, "INTERSECTID", "LONG")
    arcpy.management.CalculateField(intersectFc, "INTERSECTID", "!OBJECTID!", "PYTHON")
    arcpy.management.CalculateGeometryAttributes(intersectFc, "PIXEL_AREA AREA_GEODESIC", "", "SQUARE_METERS")
def get_and_calculate_water_pixel(intersectFc, waterfc, state):
    #Create water mask to calculate the water area
    water_mask = arcpy.analysis.Clip(intersectFc, waterfc, f"in_memory/{state}pixel_water_clipped")
    arcpy.management.CalculateGeometryAttributes(water_mask, "PIXEL_WATER AREA_GEODESIC", "", "SQUARE_METERS")
    #join the PIXEL_WATER field back to the boundary feature based on tempid
    arcpy.management.JoinField(intersectFc, "INTERSECTID", water_mask, "INTERSECTID", ["PIXEL_WATER"])
def calculate_land_pixel(intersectFc):
    #Add field PIXEL_LAND and calculate by AREA - WATER => LAND
    arcpy.AddField_management(intersectFc, "PIXEL_LAND", "FLOAT")
    with arcpy.da.UpdateCursor(intersectFc, ["PIXEL_AREA", "PIXEL_WATER", "PIXEL_LAND"]) as cursor:
            for row in cursor:
                if row[1] == None:
                    row[1] = 0 #After join there are Null values so I make it to be 0
                    cursor.updateRow(row)
                row[2] = row[0] - row[1] #GEODESIC_LAND is being calculated
                cursor.updateRow(row)
def calculate_count(intersectFc):
    #Get all the population related fields
    dens_fields = [field.name for field in arcpy.ListFields(intersectFc) if "_DENS" in field.name]
    #Calculate the densities for all those fields
    for variableField in dens_fields:
        arcpy.AddField_management(intersectFc,f"{variableField[:-5]}_CNT",'DOUBLE')
        exp = f"get_count(!{variableField}!, !PIXEL_LAND!)"

        codeblock = """def get_count(field, land):
                            if (land == 0 and field >= 0):
                                return 0
                            else:
                                return field*land

        """

        arcpy.management.CalculateField(intersectFc,f"{variableField[:-5]}_CNT",exp,"PYTHON", codeblock)
        
#Function for getting the COUNT for the fields based on the intersected pixels 
def get_stats(fishnet, intersectFc, state):
    # create list of fields to generate statistics for
    statsFields = [["PIXEL_AREA","SUM"],["PIXEL_WATER","SUM"],["PIXEL_LAND","SUM"]]
    joinFields = ["SUM_PIXEL_AREA","SUM_PIXEL_WATER","SUM_PIXEL_LAND"]
    cntFields = arcpy.ListFields(intersectFc,"*_CNT")
    if len(cntFields)>0:
        [statsFields.append([field.name,"SUM"]) for field in cntFields]
        [joinFields.append("SUM_"+f.name) for f in cntFields]
    sumTable = f"in_memory/{state}summarizeTable1"
#     fishnetInMem = arcpy.management.CopyFeatures(fishnet, "in_memory/fishnet")
    # summarize the variables for each grid cell
    arcpy.Statistics_analysis(intersectFc,sumTable,statsFields,"PIXELID")
    arcpy.JoinField_management(fishnetInMem,'PIXELID',sumTable,'PIXELID',joinFields)
    for joinField in joinFields:
        arcpy.AlterField_management(fishnetInMem,joinField,joinField.replace("SUM_",""),joinField.replace("SUM_",""))
def get_raster(fishnet, state):
    # Convert fishnet to grids
    cellSize = 0.0083333333/2 #it's used for 1km but now 500m => cellSize/2
    gridFields = ["PIXEL_LAND","PIXEL_WATER"] + [field.name for field in arcpy.ListFields(fishnet, "*_CNT")]
    raster_folder = os.mkdir(os.path.join(r"G:/usgrids2020/raster/", state))
    outFolder = f"G:/usgrids2020/raster/{state}"
    for gridField in gridFields:
        outGrid = outFolder + os.sep + state + "_" + gridField + ".tif"
        arcpy.PolygonToRaster_conversion(fishnet,gridField,outGrid,'CELL_CENTER','#',cellSize)

# the target function will be passed a function called `thread_print`
def fn(fc):
    print(f"Processing for {state[3:5].upper()}")
    readTime = datetime.datetime.now()
    arcpy.env.workspace = os.path.join(gdb_path, state)
    #Import the feature for boundary and water
    fc = f"usa{state[3:5]}_joined"
    fcInMem = arcpy.management.CopyFeatures(fc, f"in_memory/usa{state[3:5]}_fcInMem")
    water_fc = f"G:/usgrids2020/water_state/{state[3:5]}_water_layer.shp"
    waterInMem = arcpy.management.CopyFeatures(water_fc, f"in_memory/usa{state[3:5]}_waterInMem")
    add_and_calculate_geodesic_unit(fcInMem)
    get_and_calculate_water(fcInMem, waterInMem, state[3:5])
    calculate_land_area(fcInMem)
    calculate_density(fcInMem)
    print("Starting intersect fishnet to boundary...")
    fish_net = f"G:/usgrids2020/fishnets/usa{state[3:5]}_fishnet.gdb/usa{state[3:5]}_fishnet"
    fishnetInMem = arcpy.management.CopyFeatures(fish_net, f"in_memory/{state[3:5]}_fishnetInMem")
    intersectOut = f"in_memory/{state[3:5]}intersect_fc"
    arcpy.analysis.Intersect([fcInMem,fish_net], intersectOut, "NO_FID")
    add_and_calculate_geodesic_pixel(intersectOut)
    get_and_calculate_water_pixel(intersectOut, waterInMem, state[3:5])
    calculate_land_pixel(intersectOut)
    calculate_count(intersectOut)
    print("Summarizing...")
    get_stats(fishnetInMem, intersectOut, state[3:5])
    get_raster(fishnetInMem, state[3:5].upper())
    print(f"Finished processing for {state[3:5].upper()} in {str(datetime.datetime.now() - readTime)}")
        
task = ThreadWithLogAndControls(target=fn, name="do some stuff")
task.control_panel()




In [ ]:
def process(fc):
    gdb_path = "G:/usgrids2020/final_gdbs"
    gdb_list = os.listdir(gdb_path)
    gdb_list.sort()
    print(f"Processing for {state[3:5].upper()}")
    readTime = datetime.datetime.now()
    arcpy.env.workspace = os.path.join(gdb_path, state)
    #Import the feature for boundary and water
    fc = f"usa{state[3:5]}_joined"
    fcInMem = arcpy.management.CopyFeatures(fc, f"in_memory/usa{state[3:5]}_fcInMem")
    water_fc = f"G:/usgrids2020/water_state/{state[3:5]}_water_layer.shp"
    waterInMem = arcpy.management.CopyFeatures(water_fc, f"in_memory/usa{state[3:5]}_waterInMem")
     #Create the temporary id for joining fields later
    arcpy.AddField_management(fc,'tempid','LONG')
    arcpy.CalculateField_management(fc,"tempid",'!OBJECTID!','PYTHON')
    #Calculate the area(land+water) by GEODESIC method => Kytt recommended
    arcpy.management.CalculateGeometryAttributes(fc, "GEODESIC_AREA AREA_GEODESIC", "", "SQUARE_METERS")
    #Create water mask to calculate the water area
    water_mask = arcpy.analysis.Clip(fc, waterfc, f"in_memory/{state}water_clipped")
    arcpy.management.CalculateGeometryAttributes(water_mask, "GEODESIC_WATER AREA_GEODESIC", "", "SQUARE_METERS")
    #join the GEODESIC_WATER field back to the boundary feature based on tempid
    arcpy.management.JoinField(fc, "tempid", water_mask, "tempid", ["GEODESIC_WATER"])
    #Add field GEODESIC_LAND and calculate by AREA - WATER => LAND
    arcpy.AddField_management(fc, "GEODESIC_LAND", "FLOAT")
    with arcpy.da.UpdateCursor(fc, ["GEODESIC_AREA", "GEODESIC_WATER", "GEODESIC_LAND"]) as cursor:
            for row in cursor:
                if row[1] == None:
                    row[1] = 0 #After join there are Null values so I make it to be 0
                    row[2] = row[0] #GEODESIC_LAND is being calculated
                    cursor.updateRow(row)
                else:
                    row[2] = row[0] - row[1] 
                    cursor.updateRow(row)
        #Get all the population related fields
    fields = [field.name for field in arcpy.ListFields(fc)]
    new_fields = fields[35:92] + fields[93:-6]
    #Calculate the densities for all those fields
    for variableField in new_fields:
        arcpy.AddField_management(fc,f"{variableField}_DENS",'DOUBLE')
        exp = f"get_dens(!{variableField}!, !GEODESIC_LAND!)"

        codeblock = """def get_dens(field, land):
                            if (land == 0 and field >= 0):
                                return 0
                            else:
                                return field/land

        """

        arcpy.management.CalculateField(fc,f"{variableField}_DENS",exp,"PYTHON", codeblock)
    print("Starting intersect fishnet to boundary...")
    fish_net = f"G:/usgrids2020/fishnets/usa{state[3:5]}_fishnet.gdb/usa{state[3:5]}_fishnet"
    fishnetInMem = arcpy.management.CopyFeatures(fish_net, f"in_memory/{state[3:5]}_fishnetInMem")
    intersectOut = f"in_memory/{state[3:5]}intersect_fc"
    arcpy.analysis.Intersect([fcInMem,fish_net], intersectOut, "NO_FID")
    arcpy.management.AddField(intersectFc, "INTERSECTID", "LONG")
    arcpy.management.CalculateField(intersectFc, "INTERSECTID", "!OBJECTID!", "PYTHON")
    arcpy.management.CalculateGeometryAttributes(intersectFc, "PIXEL_AREA AREA_GEODESIC", "", "SQUARE_METERS")
    #Create water mask to calculate the water area
    water_mask = arcpy.analysis.Clip(intersectFc, waterfc, f"in_memory/{state}pixel_water_clipped")
    arcpy.management.CalculateGeometryAttributes(water_mask, "PIXEL_WATER AREA_GEODESIC", "", "SQUARE_METERS")
    #join the PIXEL_WATER field back to the boundary feature based on tempid
    arcpy.management.JoinField(intersectFc, "INTERSECTID", water_mask, "INTERSECTID", ["PIXEL_WATER"])
    #Add field PIXEL_LAND and calculate by AREA - WATER => LAND
    arcpy.AddField_management(intersectFc, "PIXEL_LAND", "FLOAT")
    with arcpy.da.UpdateCursor(intersectFc, ["PIXEL_AREA", "PIXEL_WATER", "PIXEL_LAND"]) as cursor:
            for row in cursor:
                if row[1] == None:
                    row[1] = 0 #After join there are Null values so I make it to be 0
                    cursor.updateRow(row)
                row[2] = row[0] - row[1] #GEODESIC_LAND is being calculated
                cursor.updateRow(row)
    #Get all the population related fields
    dens_fields = [field.name for field in arcpy.ListFields(intersectFc) if "_DENS" in field.name]
    #Calculate the densities for all those fields
    for variableField in dens_fields:
        arcpy.AddField_management(intersectFc,f"{variableField[:-5]}_CNT",'DOUBLE')
        exp = f"get_count(!{variableField}!, !PIXEL_LAND!)"

        codeblock = """def get_count(field, land):
                            if (land == 0 and field >= 0):
                                return 0
                            else:
                                return field*land

        """

        arcpy.management.CalculateField(intersectFc,f"{variableField[:-5]}_CNT",exp,"PYTHON", codeblock)
    print("Summarizing...")
    # create list of fields to generate statistics for
    statsFields = [["PIXEL_AREA","SUM"],["PIXEL_WATER","SUM"],["PIXEL_LAND","SUM"]]
    joinFields = ["SUM_PIXEL_AREA","SUM_PIXEL_WATER","SUM_PIXEL_LAND"]
    cntFields = arcpy.ListFields(intersectFc,"*_CNT")
    if len(cntFields)>0:
        [statsFields.append([field.name,"SUM"]) for field in cntFields]
        [joinFields.append("SUM_"+f.name) for f in cntFields]
    sumTable = f"in_memory/{fc}summarizeTable1"
#     fishnetInMem = arcpy.management.CopyFeatures(fishnet, "in_memory/fishnet")
    # summarize the variables for each grid cell
    arcpy.Statistics_analysis(intersectFc,sumTable,statsFields,"PIXELID")
    arcpy.JoinField_management(fishnet,'PIXELID',sumTable,'PIXELID',joinFields)
    for joinField in joinFields:
        arcpy.AlterField_management(fishnet,joinField,joinField.replace("SUM_",""),joinField.replace("SUM_",""))
    # Convert fishnet to grids
    cellSize = 0.0083333333/2 #it's used for 1km but now 500m => cellSize/2
    gridFields = ["PIXEL_LAND","PIXEL_WATER"] + [field.name for field in arcpy.ListFields(fishnet, "*_CNT")]
    raster_folder = os.mkdir(os.path.join(r"G:/usgrids2020/raster/", state))
    outFolder = f"G:/usgrids2020/raster/{state}"
    for gridField in gridFields:
        outGrid = outFolder + os.sep + state + "_" + gridField + ".tif"
        arcpy.PolygonToRaster_conversion(fishnet,gridField,outGrid,'CELL_CENTER','#',cellSize)
    print(f"Finished processing for {state[3:5].upper()} in {str(datetime.datetime.now() - readTime)}")


